## New dataset
1. List of ids from jsons
2. Retrieve metadata from manifest file
3. Create dataframe wih timestamps

In [3]:
import pandas as pd
import os, sys, json
import numpy as np
from typing import Optional, List, Tuple
from tqdm import tqdm
rng = np.random.default_rng()
sys.path.append('../')
from utils import subdivide_dir, NLAUVI_LOC_C, NLAUVI_LOC_N

In [6]:
base = '../../../../data/samenvattingen/'
base_csv = '../../../../data/samenvattingen/ArchiefExport_'
years = ['2019', '2020']
# path_csv = '../../../data/samenvattingen/ArchiefExport_2019.csv'
# path_2019 = "../../../data/samenvattingen/2019"

### Process manifest file

In [38]:
# Get IDs & file location from processed files
dfs = []
for year in years:
    id_dict = {}
    for root, _, files in os.walk(base + year):
        for file in files:
            if file.endswith(".json"):
                id_dict[file.split(".")[0]] = os.path.join(root, file)

    # Retrieve only processed IDs
    df = pd.read_csv(base_csv + year + ".csv")
    df = df[df["ID"].isin(id_dict.keys())].copy()
    df.reset_index(drop=True, inplace=True)
    df['Path'] = df['ID'].apply(lambda id: id_dict[id])
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df.reset_index(drop=True, inplace=True)
# Drop irrelevant columns
df = df.drop(columns=["AlertTime", "SpeechUrl", "MediaUrl", "Summary"], errors='ignore')
del dfs
print(f'Number of processed files: {len(df)}')

Number of processed files: 111232


In [8]:
# Util functions
def get_ms(timestamp):
   """
   Transform timestamp from minutes into miliseconds
   """
   ms = 0
   timestamp = timestamp.split(".")[0]
   timestamp = timestamp.split(":")
   ms += (int(timestamp[1]) * 60 + int(timestamp[2])) * 1000
   return ms
  
def adj_start_ms(x):
   """"
   Recording start ~5 minutes (30000ms) before segment starts, but recorded times can be any time within a broadcast.
   This shifts start times to 5 min or less. Extra 10000ms is added for robustness.
   """
   if x > 300000:
       return 310000
   else:
       return x

In [39]:
# Adjust timings and create duration column
if "StartPosition" in df.keys():
    start_ms = df["StartPosition"].apply(get_ms)
    end_ms = df["EndPosition"].apply(get_ms)
    df["duration_ms"] = end_ms - start_ms
    df["adj_start_ms"] = start_ms.apply(lambda x: adj_start_ms(x))
    df["adj_end_ms"] = df["adj_start_ms"] + df["duration_ms"]
    df = df.drop(columns=["StartPosition", "EndPosition", "start_ms", "end_ms"], errors="ignore")
# Drop rows with durations of 0
df = df[df["duration_ms"] > 0]
df.head(5)

,ID,ProgrammeDateTime,Path,duration_ms,adj_start_ms,adj_end_ms
0,E94B5735-A16D-41A9-A05A-09EE40E240A3,2019-01-01 05:00:00.000,../../../../data/samenvattingen/2019/01/01/E94...,331000,310000,641000
1,168D1E30-8F21-4C9D-A5DD-149D77E233B1,2019-01-01 06:00:00.000,../../../../data/samenvattingen/2019/01/01/168...,17000,137000,154000
2,FCDD8B88-9BC5-4F84-BA71-610BFA889314,2019-01-01 06:00:00.000,../../../../data/samenvattingen/2019/01/01/FCD...,17000,163000,180000
3,9F461187-A082-42DF-B09B-7566B290657F,2019-01-01 06:00:00.000,../../../../data/samenvattingen/2019/01/01/9F4...,486000,310000,796000
4,3376B065-A0BD-4E3D-AC05-14B278077F90,2019-01-01 07:00:00.000,../../../../data/samenvattingen/2019/01/01/337...,333000,310000,643000


### Parse the transcripts from the json files
- Only relevant transcripts for summaries
- parse from "text" ipv "result" 

In [30]:
"""
Dataset JSON loader
Recordings are divided into sentences like so:
[
    {
        "result": [
            [
                "word",
                starttime,
                endtime,
                confidence
            ],
            ...
        ],
        "text": entire sentence,
        "speaker": "unk"??
    },
    ...
]
""" 
def get_index(data, start, end) -> Tuple[int, int]:
    # Get the beginning of sentence times
    bos_times = []
    for i in range(len(data)):
        bos_times.append(data[i]['result'][0][1])
    
    # Find bos_time closest to start/end and return the index
    if len(bos_times) > 0:
        s = min(bos_times, key=lambda x:abs(x-start))
        e = min(bos_times, key=lambda x:abs(x-end))
        return bos_times.index(s), bos_times.index(e)
    else:
        return 0, 0

def get_text_from_index(transcript, index):
    """
    Get the relevant segment out of a full transcript
    """
    start_index, end_index = index[0], index[1]
    return transcript[start_index:end_index]

In [40]:
# Get the full transcript out of the .json file, also extract the relevant segment out of it.
# Takes ~10 minutes.
t_full = []
t_sub = []
t_indices = []
for i, path in enumerate(df.Path):
    with open(path) as f:
        data = json.load(f)
    index = get_index(data, df.adj_start_ms.iloc[i], df.adj_end_ms.iloc[i])
    transcript = [t['text'] for t in data]
    t_full.append(transcript)
    t_sub.append(get_text_from_index(transcript, index))
    t_indices.append(index)

df['Transcript_full'] = t_full
df['Transcript_sub'] = t_sub
df['Transcript_indices'] = t_indices

In [41]:
# Final dataframe cleaned from (near) empty transcripts
empty = df[df.Transcript_sub.apply(lambda x: len(x) < 4)]
df.drop(empty.index, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

# Check for empty transcripts
# uncomment to save, already saved in csv!
# dfc.to_csv("/AI/data/samenvattingen/csv/data_2019_04_clean.csv", index=False)

,ID,ProgrammeDateTime,Path,duration_ms,adj_start_ms,adj_end_ms,Transcript_full,Transcript_sub,Transcript_indices
0,E94B5735-A16D-41A9-A05A-09EE40E240A3,2019-01-01 05:00:00.000,../../../../data/samenvattingen/2019/01/01/E94...,331000,310000,641000,[Straat waren rode cirkels getekend en niemand...,"[Nou ja goed, we gaan eens even kijken., Peter...","(34, 86)"
1,9F461187-A082-42DF-B09B-7566B290657F,2019-01-01 06:00:00.000,../../../../data/samenvattingen/2019/01/01/9F4...,486000,310000,796000,[Een zinzen St Regis was dat is kwart over zes...,[André Meinema van de economieredactie die hee...,"(41, 105)"
2,3376B065-A0BD-4E3D-AC05-14B278077F90,2019-01-01 07:00:00.000,../../../../data/samenvattingen/2019/01/01/337...,333000,310000,643000,[En zeggen we waar we zijn en wat er aan de ha...,[Zeventien vuurwerk en oud en nieuw onlosmakel...,"(72, 125)"
3,906D9A09-6E9B-4B01-ADEB-0A13D29D4A71,2019-01-01 07:00:00.000,../../../../data/samenvattingen/2019/01/01/906...,355000,259000,614000,[In tweeduizendnegentien reden we nog op benzi...,[Daar is de brandweer nog steeds bezig met het...,"(48, 126)"
4,BEDD587D-DC23-45BE-A3C5-8BFBB9FCBDE6,2019-01-01 08:00:00.000,../../../../data/samenvattingen/2019/01/01/BED...,33000,182000,215000,[Een reis op krans .nl ontdek Marokko met nu t...,[Het nieuwe jaar betekent ook dat er een aanta...,"(37, 43)"


In [42]:
print(f'Number of segments: {len(df)}')
print(f'Avg. segment length: {np.mean(df.Transcript_sub.apply(lambda x: len(x))):.4}')

Number of segments: 98530
Avg. segment length: 48.92


### Concatenated Dataset

In [42]:
MIN_SECTIONS = 2
MAX_SECTIONS = 5

def create_docs(data: pd.DataFrame, save=True) -> Optional[List[str]]:
    """
    Creates a folder containing .txt files from concatenated segments.
    :param data: Dataframe containing the data to be processed.
    :param save: Whether to save to a folder or to return the results as a list of Strings.
    """
    d = data.copy()
    i = 0
    pad = int(np.log10(len(d))) + 1

    if save:
        os.makedirs(NLAUVI_LOC_C, exist_ok=True)
    else:
        docs = []

    with tqdm(total=len(d), desc='Transcripts processed') as pbar:
        while len(d) > 0:
            if len(d) > MAX_SECTIONS + 1: # +1 to prevent being left with a single segment
                n = rng.integers(MIN_SECTIONS, MAX_SECTIONS + 1)
            elif len(d) > MAX_SECTIONS:
                n = rng.integers(MIN_SECTIONS, MAX_SECTIONS)
            else:
                n = len(d)
            slice = rng.choice(len(d), size=n, replace=False)

            doc = ''
            l = 0
            for t in slice:
                text = d["Transcript_sub"].iloc[t]
                doc += f'==={d["ID"].iloc[t]}===\n{" ".join(text)}\n'
                l += len(text)

            d.drop(slice, inplace=True)
            d.reset_index(drop=True, inplace=True)

            if save:
                with open(f'{NLAUVI_LOC_C}/{i:0{pad}}', 'w') as tfile:
                    tfile.write(doc)
            else:
                docs.append(doc)
            i += 1
            pbar.update(n)

    print(f'Created {i} Documents')

    if save is False: 
        return docs

In [43]:
create_docs(df)
subdivide_dir(NLAUVI_LOC_C)

Transcripts processed: 100%|██████████| 98530/98530 [04:30<00:00, 363.76it/s] 

Created 28217 Documents
   Average document length = 48.88778218804267 sentences


### Normal Dataset
Unconcatenated dataset

In [51]:
def create_docs_normal(data: pd.DataFrame, n: Optional[int] = None, save=True) -> Optional[List[str]]:
    """
    Creates a folder containing .txt files from unaltered segments.
    :param data: Dataframe containing the data to be processed.
    :param save: Whether to save to a folder or to return the results as a list of Strings.
    """
    d = data.copy()
    if n:
        d = d[:n]
    pad = int(np.log10(len(d))) + 1

    if save:
        os.makedirs('../'+NLAUVI_LOC_N, exist_ok=True)
    else:
        docs = []

    pbar = tqdm(d.itertuples(), total=len(d), desc='Transcripts processed')
    for i, row in enumerate(pbar):
        doc = ''
        sections = []
        sections.append(' '.join(row.Transcript_full[0:row.Transcript_indices[0]]))
        sections.append(' '.join(row.Transcript_full[row.Transcript_indices[0]:row.Transcript_indices[1]]))
        sections.append(' '.join(row.Transcript_full[row.Transcript_indices[1]:]))
        if len(sections) < 2:
            raise ValueError('Less than 2 sections')
        for s in sections:
            doc += f'==={row.ID}===\n{s}\n'
        
        if save:
            with open(f'../{NLAUVI_LOC_N}/{i:0{pad}}', 'w') as tfile:
                tfile.write(doc)
        else:
            docs.append(doc)

    print(f'Created {i} Documents')

    if save is False: 
        return docs

In [52]:
create_docs_normal(df)
subdivide_dir('../'+NLAUVI_LOC_N)

Transcripts processed: 100%|██████████| 98530/98530 [00:07<00:00, 13293.08it/s]


Created 98529 Documents
